In [1]:
%load_ext autoreload
# Enable autoreload for all modules
%autoreload 2

In [2]:
from iris_environments.environments import get_environment_builder, get_robot_instance_names
from cspace_utils.graphs import compute_greedy_clique_partition
from cspace_utils.plotting import (plot_cliuqes, 
                                   plot_visibility_graph, 
                                   plot_edges, 
                                   plot_points,
                                   plot_2d_ellipse_meshcat,
                                   plot_vpoly_2d_meshcat)
from cspace_utils.environment_helpers import get_col_func, get_coverage_estimator, get_sample_cfree_handle, vgraph
import numpy as np
import ipywidgets as widgets
from functools import partial
from pydrake.all import (RigidTransform, Rgba, Sphere, RotationMatrix, SceneGraphCollisionChecker)

In [3]:
from iris_environments.environments import env_names
currname = '2DOFBLOCKS'
plant_builder = get_environment_builder(currname)
plant, scene_graph, diagram, diagram_context, plant_context, meshcat = plant_builder(usemeshcat=True)

scene_graph_context = scene_graph.GetMyMutableContextFromRoot(
    diagram_context)

rob_names = get_robot_instance_names(currname)
robot_instances = [plant.GetModelInstanceByName(n) for n in rob_names]
checker = SceneGraphCollisionChecker(model = diagram, 
                    robot_model_instances = robot_instances,
                    #configuration_distance_function = _configuration_distance,
                    edge_step_size = 0.125)

INFO:drake:Meshcat listening for connections at http://localhost:7000
INFO:drake:Allocating contexts to support implicit context parallelism 20


http://localhost:7000


In [4]:
col_hand=get_col_func(plant, plant_context, use_ik=False)
sample_cfree = get_sample_cfree_handle(plant.GetPositionLowerLimits(), 
                        plant.GetPositionUpperLimits(),
                        col_hand)

In [5]:
pts,_ = sample_cfree(50, 1000,[])
plot_points(meshcat, pts, 'vgraph/points', size=0.01)
ad_mat = vgraph(pts, checker=checker, parallelize=True)

In [6]:
from pydrake.all import Hyperellipsoid, VPolytope
cliques = compute_greedy_clique_partition(ad_mat.toarray(), min_cliuqe_size=10)

plot_cliuqes(meshcat, cliques, pts, 'cliques', downsampling=5)

clique_pts = [pts[c] for c in cliques]
ellipsoids = [Hyperellipsoid.MinimumVolumeCircumscribedEllipsoid(p.T) for p in clique_pts if len(p)>4]
for i, e in enumerate(ellipsoids):
    plot_2d_ellipse_meshcat(meshcat, e, f"cliques/ellipsoids/e_{i}", size =0.01)
  

vpolys = [VPolytope(c.T).GetMinimalRepresentation() for c in clique_pts]
for i, v in enumerate(vpolys):
    plot_vpoly_2d_meshcat(meshcat, v, f"cliques/vpolys/v_{i}", size =0.015)

Set parameter Username
Academic license - for non-commercial use only - expires 2024-06-11


In [11]:
vp = vpolys[0]

In [ ]:
vp.

In [5]:
#make configuration file 
# import os
# point_file = 'benchmarks/visibility_graphs/'+currname+'_pts.yml'
# if point_file.split('/')[2] in os.listdir('benchmarks/seedpoints'):
#     file = open(point_file, 'a') 
# else:
#     file = open(point_file, 'w')
#     file.write('points:\n')

FileNotFoundError: [Errno 2] No such file or directory: 'benchmarks/visibility_graphs/2DOFBLOCKS_pts.yml'

In [ ]:
q = np.zeros(plant.num_positions()) 
sliders = []
for i in range(plant.num_positions()):
    q_low = plant.GetPositionLowerLimits()[i]*0.99
    q_high = plant.GetPositionUpperLimits()[i]*0.99
    sliders.append(widgets.FloatSlider(min=q_low, max=q_high, value=0, step=0.001, description=f"q{i}"))

col_col =  Rgba(0.8, 0.0, 0, 0.5)    
col_free =  Rgba(0.0, 0.8, 0.5, 0.5) 
def showres(qvis):
    plant.SetPositions(plant_context, qvis)
    diagram.ForcedPublish(diagram_context)
    query = plant.get_geometry_query_input_port().Eval(plant_context)
    col = query.HasCollisions()
    if col:
        meshcat.SetObject(f"/drake/visualizer/shunk",
                                   Sphere(0.2),
                                   col_col)
    else:
        meshcat.SetObject(f"/drake/visualizer/shunk",
                                   Sphere(0.2),
                                   col_free)
    meshcat.SetTransform(f"/drake/visualizer/shunk",
                                   RigidTransform(RotationMatrix(),
                                                  np.array([0,0,2])))
    return col

def handle_slider_change(change, idx):
    q[idx] = change['new']
    showres(q)
idx = 0
for slider in sliders:
    slider.observe(partial(handle_slider_change, idx = idx), names='value')
    idx+=1

for slider in sliders:
    display(slider)

def write_seed_point_to_file(button):
    col = showres(q)
    if not col:
        line  = '- ['
        for a in q[:-1]:
            line+= str(a)+', '
        line+= str(q[-1])+']\n'
        file.write(line)
        file.flush()
        #write seedpoint to file
    else:
        raise ValueError("That point is in collision")

button = widgets.Button(description="Save")

# Attach the function to the button's click event
button.on_click(write_seed_point_to_file)

# Display the button
display(button)